<a href="https://colab.research.google.com/github/santiagonajera/OptimizacionRutas/blob/main/OptimoTransporte_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.optimize import linprog

# Definimos los costos de envío (en forma de vector)
# Orden: CD1->C1, CD1->C2, CD1->C3,
#        CD2->C1, CD2->C2, CD2->C3,
#        CD3->C1, CD3->C2, CD3->C3
costos = np.array([
    [5, 4, 3],
    [8, 4, 3],
    [9, 7, 5]
]).flatten()

# Vector objetivo (minimizar costo total)
c = costos

# Variables de decisión: x_ij = cantidad enviada desde CD i a Ciudad j
# Hay 3x3 = 9 variables
# x[0] = CD1 -> C1, x[1] = CD1 -> C2, x[2] = CD1 -> C3
# x[3] = CD2 -> C1, x[4] = CD2 -> C2, x[5] = CD2 -> C3
# x[6] = CD3 -> C1, x[7] = CD3 -> C2, x[8] = CD3 -> C3

# Restricciones de oferta (sumatoria por fila <= oferta)
A_ub = []
b_ub = []

# Oferta de cada centro de distribución
oferta = np.array([200, 400, 300])
for i in range(3):
    row = np.zeros(9)
    for j in range(3):
        row[i * 3 + j] = 1  # Variable x_ij
    A_ub.append(row)
    b_ub.append(oferta[i])

# Demanda de cada ciudad (sumatoria por columna >= demanda)
demanda = np.array([500, 200, 200])
for j in range(3):
    row = np.zeros(9)
    for i in range(3):
        row[i * 3 + j] = 1  # Variable x_ij
    A_ub.append(row)
    b_ub.append(demanda[j])

# Ahora, como linprog maneja desigualdades <=, convertimos las demandas >= a <=
# Pero como no se puede usar >= directamente, usamos:
# Para demanda: sum(x_i,j) >= demanda_j → -sum(x_i,j) <= -demanda_j
# Entonces agregamos restricciones negativas
for j in range(3):
    row = np.zeros(9)
    for i in range(3):
        row[i * 3 + j] = -1
    A_ub.append(row)
    b_ub.append(-demanda[j])

# También debemos asegurar que todas las variables sean no negativas
bounds = [(0, None) for _ in range(9)]

# Resolver el problema
result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')

# Mostrar resultados
if result.success:
    print("✅ Problema resuelto exitosamente.")
    print(f"Costo mínimo total: ${result.fun:.2f}")

    # Reorganizar soluciones en matriz
    x = result.x.reshape((3, 3))
    print("\nCantidad enviada (CD -> Ciudad):")
    print("           Ciudad 1     Ciudad 2     Ciudad 3")
    print("CD 1      ", x[0,0], "         ", x[0,1], "         ", x[0,2])
    print("CD 2      ", x[1,0], "         ", x[1,1], "         ", x[1,2])
    print("CD 3      ", x[2,0], "         ", x[2,1], "         ", x[2,2])
else:
    print("❌ No se pudo encontrar una solución factible.")

✅ Problema resuelto exitosamente.
Costo mínimo total: $5100.00

Cantidad enviada (CD -> Ciudad):
           Ciudad 1     Ciudad 2     Ciudad 3
CD 1       200.0           0.0           0.0
CD 2       -0.0           200.0           200.0
CD 3       300.0           0.0           0.0


In [ ]:
import numpy as np
from scipy.optimize import linprog

# -----------------------------
# Datos de entrada (pueden cargarse desde archivos o entradas)
# -----------------------------

# Matriz de costos: [centro][ciudad]
costos = np.array([
    [5.00, 4.00, 3.00],  # CD1 → C1, C2, C3
    [8.00, 4.00, 3.00],  # CD2 → C1, C2, C3
    [9.00, 7.00, 5.00]   # CD3 → C1, C2, C3
])

# Demanda por ciudad (vector)
demanda = np.array([500, 200, 200])  # Ciudad1, Ciudad2, Ciudad3

# Oferta por centro de distribución (vector)
oferta = np.array([200, 400, 300])  # CD1, CD2, CD3

# -----------------------------
# Parámetros del problema
# -----------------------------
num_centros = len(oferta)
num_ciudades = len(demanda)

# Verificación de dimensiones
if costos.shape != (num_centros, num_ciudades):
    raise ValueError("La matriz de costos debe tener dimensiones (num_centros, num_ciudades)")

# -----------------------------
# Preparación del modelo de optimización
# -----------------------------

# Vector objetivo: costo total
c = costos.flatten()

# Restricciones de igualdad: oferta y demanda
A_eq = []
b_eq = []

# 1. Restricciones de oferta: suma de envíos desde cada centro = oferta
for i in range(num_centros):
    row = np.zeros(num_centros * num_ciudades)
    for j in range(num_ciudades):
        row[i * num_ciudades + j] = 1  # x[i,j]
    A_eq.append(row)
    b_eq.append(oferta[i])

# 2. Restricciones de demanda: suma de envíos a cada ciudad = demanda
for j in range(num_ciudades):
    row = np.zeros(num_centros * num_ciudades)
    for i in range(num_centros):
        row[i * num_ciudades + j] = 1  # x[i,j]
    A_eq.append(row)
    b_eq.append(demanda[j])

# Límites inferiores: todas las variables >= 0
bounds = [(0, None) for _ in range(num_centros * num_ciudades)]

# Resolver el problema de transporte
result = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

# Mostrar resultados
if result.success:
    print("✅ Problema resuelto exitosamente.")
    print(f"Costo mínimo total: ${result.fun:.2f}")

    # Reorganizar la solución en una matriz
    x = result.x.reshape((num_centros, num_ciudades))

    print("\nCantidad enviada (Centro → Ciudad):")
    print("           ", end="")
    for j in range(num_ciudades):
        print(f"Ciudad{j+1:>8}", end="")
    print()
    print("-" * 40)

    for i in range(num_centros):
        print(f"CD{i+1:<8}", end="")
        for j in range(num_ciudades):
            print(f"{x[i,j]:>10.0f}", end="")
        print()
else:
    print("❌ No se pudo encontrar una solución factible.")

✅ Problema resuelto exitosamente.
Costo mínimo total: $5100.00

Cantidad enviada (Centro → Ciudad):
           Ciudad       1Ciudad       2Ciudad       3
----------------------------------------
CD1              200         0         0
CD2                0       200       200
CD3              300         0        -0


In [ ]:
import numpy as np
from scipy.optimize import linprog
import pandas as pd

# -----------------------------
# Cargar datos desde Excel en línea
# -----------------------------
url = "https://github.com/santiagonajera/OptimizacionRutas/raw/refs/heads/main/OptimoTransportes2.xlsx"

# Leer las hojas del archivo Excel
try:
    # Leer hoja 'COSTOS'
    df_costos = pd.read_excel(url, sheet_name='COSTOS', index_col=0)
    costos = df_costos.values  # Convertir a NumPy array

    # Leer hoja 'DEMANDA'
    df_demanda = pd.read_excel(url, sheet_name='DEMANDA')
    demanda = df_demanda['DEMANDA'].values  # Extraer columna DEMANDA

    # Leer hoja 'OFERTA'
    df_oferta = pd.read_excel(url, sheet_name='OFERTA')
    oferta = df_oferta['OFERTA'].values  # Extraer columna OFERTA

    print("✅ Datos cargados correctamente desde el archivo Excel.")

except Exception as e:
    print(f"❌ Error al cargar el archivo Excel: {e}")
    exit()

# -----------------------------
# Parámetros del problema
# -----------------------------
num_centros = len(oferta)
num_ciudades = len(demanda)

# Verificación de dimensiones
if costos.shape != (num_centros, num_ciudades):
    raise ValueError(f"La matriz de costos debe tener tamaño ({num_centros}, {num_ciudades}), pero tiene {costos.shape}")

# -----------------------------
# Preparación del modelo de optimización
# -----------------------------

# Vector objetivo: costo total
c = costos.flatten()

# Restricciones de igualdad: oferta y demanda
A_eq = []
b_eq = []

# 1. Restricciones de oferta: suma de envíos desde cada centro = oferta
for i in range(num_centros):
    row = np.zeros(num_centros * num_ciudades)
    for j in range(num_ciudades):
        row[i * num_ciudades + j] = 1  # x[i,j]
    A_eq.append(row)
    b_eq.append(oferta[i])

# 2. Restricciones de demanda: suma de envíos a cada ciudad = demanda
for j in range(num_ciudades):
    row = np.zeros(num_centros * num_ciudades)
    for i in range(num_centros):
        row[i * num_ciudades + j] = 1  # x[i,j]
    A_eq.append(row)
    b_eq.append(demanda[j])

# Límites inferiores: todas las variables >= 0
bounds = [(0, None) for _ in range(num_centros * num_ciudades)]

# Resolver el problema de transporte
result = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

# Mostrar resultados
if result.success:
    print("\n✅ Problema resuelto exitosamente.")
    print(f"Costo mínimo total: ${result.fun:.2f}")

    # Reorganizar la solución en una matriz
    x = result.x.reshape((num_centros, num_ciudades))

    print("\nCantidad enviada (Centro → Ciudad):")
    print("           ", end="")
    for j in range(num_ciudades):
        print(f"Ciudad{j+1:>8}", end="")
    print()
    print("-" * 40)

    for i in range(num_centros):
        print(f"CD{i+1:<8}", end="")
        for j in range(num_ciudades):
            print(f"{x[i,j]:>10.0f}", end="")
        print()
else:
    print("\n❌ No se pudo encontrar una solución factible.")
    print(result.message)

✅ Datos cargados correctamente desde el archivo Excel.

❌ No se pudo encontrar una solución factible.
The problem is infeasible. (HiGHS Status 8: model_status is Infeasible; primal_status is Infeasible)


In [ ]:
import numpy as np
from scipy.optimize import linprog
import pandas as pd

# -----------------------------
# Configuración
# -----------------------------
url = "https://github.com/santiagonajera/OptimizacionRutas/raw/refs/heads/main/OptimoTransportes.xlsx"
hojas = ['COSTOS', 'DEMANDA', 'OFERTA']

print("📥 Cargando datos desde el archivo Excel...")

# -----------------------------
# Cargar datos desde Excel
# -----------------------------
try:
    # Leer COSTOS (matriz: centros × ciudades)
    df_costos = pd.read_excel(url, sheet_name='COSTOS', index_col=0)
    costos = df_costos.values
    centros = df_costos.index.tolist()  # Nombres: CD1, CD2, ...
    ciudades = df_costos.columns.tolist()  # Nombres: Ciudad1, Ciudad2, ...

    # Leer DEMANDA (debe tener una columna 'DEMANDA' y nombres de ciudades)
    df_demanda = pd.read_excel(url, sheet_name='DEMANDA')
    if 'CIUDAD' not in df_demanda.columns or 'DEMANDA' not in df_demanda.columns:
        raise ValueError("La hoja 'DEMANDA' debe tener columnas 'CIUDAD' y 'DEMANDA'")
    demanda_ciudad = df_demanda.set_index('CIUDAD')['DEMANDA'].reindex(ciudades, fill_value=0)
    demanda = demanda_ciudad.values

    # Leer OFERTA (debe tener 'CENTRO_DIST' y 'OFERTA')
    df_oferta = pd.read_excel(url, sheet_name='OFERTA')
    if 'CENTRO_DIST' not in df_oferta.columns or 'OFERTA' not in df_oferta.columns:
        raise ValueError("La hoja 'OFERTA' debe tener columnas 'CENTRO_DIST' y 'OFERTA'")
    oferta_centro = df_oferta.set_index('CENTRO_DIST')['OFERTA'].reindex(centros, fill_value=0)
    oferta = oferta_centro.values

    print(f"✅ Datos cargados correctamente.")
    print(f"   → {len(centros)} centros: {centros}")
    print(f"   → {len(ciudades)} ciudades: {ciudades}")

except Exception as e:
    print(f"❌ Error al cargar los datos: {e}")
    exit()

# -----------------------------
# Validar consistencia
# -----------------------------
if costos.shape != (len(centros), len(ciudades)):
    print(f"❌ Error: la matriz de costos tiene tamaño {costos.shape}, pero se esperaba ({len(centros)}, {len(ciudades)})")
    exit()

if not np.isclose(np.sum(oferta), np.sum(demanda)):
    print(f"⚠️ Advertencia: Oferta total ({np.sum(oferta)}) ≠ Demanda total ({np.sum(demanda)})")
    print("   → El problema puede ser infactible si no se balancea.")
    # Puedes agregar un centro ficticio o ciudad ficticia si es necesario
else:
    print(f"✅ Oferta total = Demanda total = {np.sum(oferta)} → Problema balanceado")

# -----------------------------
# Preparar problema de transporte
# -----------------------------
num_centros = len(centros)
num_ciudades = len(ciudades)

c = costos.flatten()  # Vector de costos

A_eq = []
b_eq = []

# 1. Restricciones de oferta: envíos desde cada centro = oferta[i]
for i in range(num_centros):
    row = np.zeros(num_centros * num_ciudades)
    for j in range(num_ciudades):
        row[i * num_ciudades + j] = 1
    A_eq.append(row)
    b_eq.append(oferta[i])

# 2. Restricciones de demanda: envíos a cada ciudad = demanda[j]
for j in range(num_ciudades):
    row = np.zeros(num_centros * num_ciudades)
    for i in range(num_centros):
        row[i * num_ciudades + j] = 1
    A_eq.append(row)
    b_eq.append(demanda[j])

# Límites: x >= 0
bounds = [(0, None)] * (num_centros * num_ciudades)

# Resolver
print("\n🔍 Resolviendo el problema de transporte...")
result = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

# -----------------------------
# Mostrar resultados
# -----------------------------
if result.success:
    print("\n✅ Solución óptima encontrada.")
    print(f"💰 Costo total mínimo: ${result.fun:,.2f}")

    # Solución en forma de matriz
    x = result.x.reshape((num_centros, num_ciudades))

    print(f"\n📦 Cantidad enviada (unidades):")
    print("           ", end="")
    for ciudad in ciudades:
        print(f"{ciudad:>10}", end="")
    print()
    print("-" * (12 + 10 * num_ciudades))

    for i, centro in enumerate(centros):
        print(f"{centro:<10}", end="")
        for j in range(num_ciudades):
            print(f"{x[i,j]:>10.0f}", end="")
        print()

else:
    print("\n❌ No se encontró una solución factible.")
    print(f"Motivo: {result.message}")

📥 Cargando datos desde el archivo Excel...
✅ Datos cargados correctamente.
   → 3 centros: ['CD 1', 'CD 2', 'CD 3']
   → 3 ciudades: ['Ciudad 1', 'Ciudad 2', 'Ciudad 3']
⚠️ Advertencia: Oferta total (0) ≠ Demanda total (900)
   → El problema puede ser infactible si no se balancea.

🔍 Resolviendo el problema de transporte...

❌ No se encontró una solución factible.
Motivo: The problem is infeasible. (HiGHS Status 8: model_status is Infeasible; primal_status is None)


In [ ]:
import numpy as np
from scipy.optimize import linprog
import pandas as pd

# -----------------------------
# Configuración
# -----------------------------
url = "https://github.com/santiagonajera/OptimizacionRutas/raw/refs/heads/main/OptimoTransportes.xlsx"

print("📥 Cargando datos desde el archivo Excel...")

# -----------------------------
# Función para limpiar nombres (quitar espacios, minúsculas)
# -----------------------------
def limpiar_nombre(nombre):
    if isinstance(nombre, str):
        return nombre.strip().replace(" ", "").lower()
    return str(nombre).strip().replace(" ", "").lower()

# -----------------------------
# Cargar datos
# -----------------------------
try:
    # Leer COSTOS
    df_costos = pd.read_excel(url, sheet_name='COSTOS', index_col=0)
    # Limpiar índices y columnas
    centros_raw = df_costos.index.tolist()
    ciudades_raw = df_costos.columns.tolist()

    centros = [limpiar_nombre(c) for c in centros_raw]
    ciudades = [limpiar_nombre(c) for c in ciudades_raw]

    costos = df_costos.values

    # Leer DEMANDA
    df_demanda = pd.read_excel(url, sheet_name='DEMANDA')
    if 'CIUDAD' not in df_demanda.columns or 'DEMANDA' not in df_demanda.columns:
        raise ValueError("Faltan columnas 'CIUDAD' o 'DEMANDA' en hoja DEMANDA")

    demanda_ciudad = df_demanda['CIUDAD'].apply(limpiar_nombre)
    demanda_valores = df_demanda['DEMANDA'].values
    demanda_dict = dict(zip(demanda_ciudad, demanda_valores))

    # Alinear con ciudades en COSTOS
    demanda = np.array([demanda_dict.get(c, 0) for c in ciudades])

    # Leer OFERTA
    df_oferta = pd.read_excel(url, sheet_name='OFERTA')
    if 'CENTRO_DIST' not in df_oferta.columns or 'OFERTA' not in df_oferta.columns:
        raise ValueError("Faltan columnas 'CENTRO_DIST' u 'OFERTA' en hoja OFERTA")

    centro_dist = df_oferta['CENTRO_DIST'].apply(limpiar_nombre)
    oferta_valores = df_oferta['OFERTA'].values
    oferta_dict = dict(zip(centro_dist, oferta_valores))

    # Alinear con centros en COSTOS
    oferta = np.array([oferta_dict.get(c, 0) for c in centros])

    print(f"✅ Datos cargados correctamente.")
    print(f"   → Centros: {centros_raw}")
    print(f"   → Ciudades: {ciudades_raw}")

except Exception as e:
    print(f"❌ Error al cargar los datos: {e}")
    exit()

# -----------------------------
# Validación de tamaño
# -----------------------------
if costos.shape != (len(centros), len(ciudades)):
    print(f"❌ Error: matriz COSTOS es {costos.shape}, esperado ({len(centros)}, {len(ciudades)})")
    exit()

oferta_total = np.sum(oferta)
demanda_total = np.sum(demanda)

print(f"📊 Oferta total: {oferta_total}")
print(f"📊 Demanda total: {demanda_total}")

if not np.isclose(oferta_total, demanda_total):
    print(f"❌ Oferta ≠ Demanda: el problema puede ser infactible.")
    print("💡 Solución: balancear con centro o ciudad ficticia.")
    exit()
else:
    print("✅ Problema balanceado → se puede resolver.")

# -----------------------------
# Resolver problema
# -----------------------------
num_centros = len(centros)
num_ciudades = len(ciudades)
c = costos.flatten()

A_eq = []
b_eq = []

# Oferta
for i in range(num_centros):
    row = np.zeros(num_centros * num_ciudades)
    for j in range(num_ciudades):
        row[i * num_ciudades + j] = 1
    A_eq.append(row)
    b_eq.append(oferta[i])

# Demanda
for j in range(num_ciudades):
    row = np.zeros(num_centros * num_ciudades)
    for i in range(num_centros):
        row[i * num_ciudades + j] = 1
    A_eq.append(row)
    b_eq.append(demanda[j])

bounds = [(0, None)] * (num_centros * num_ciudades)

print("\n🔍 Resolviendo...")
result = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

# -----------------------------
# Mostrar resultados
# -----------------------------
if result.success:
    print("\n✅ Solución encontrada.")
    print(f"💰 Costo mínimo: ${result.fun:,.2f}")

    x = result.x.reshape(num_centros, num_ciudades)

    print(f"\n📦 Envíos (unidades):")
    print(f"{'':<8}" + "".join([f"{ciudad:>10}" for ciudad in ciudades_raw]))
    print("-" * 50)
    for i, centro in enumerate(centros_raw):
        print(f"{centro:<8}" + "".join([f"{x[i,j]:>10.0f}" for j in range(num_ciudades)]))
else:
    print("\n❌ No se encontró solución factible.")
    print(f"🔧 Mensaje: {result.message}")

📥 Cargando datos desde el archivo Excel...
✅ Datos cargados correctamente.
   → Centros: ['CD 1', 'CD 2', 'CD 3']
   → Ciudades: ['Ciudad 1', 'Ciudad 2', 'Ciudad 3']
📊 Oferta total: 900
📊 Demanda total: 900
✅ Problema balanceado → se puede resolver.

🔍 Resolviendo...

✅ Solución encontrada.
💰 Costo mínimo: $5,100.00

📦 Envíos (unidades):
          Ciudad 1  Ciudad 2  Ciudad 3
--------------------------------------------------
CD 1           200         0         0
CD 2             0       200       200
CD 3           300         0        -0


In [1]:
import numpy as np
from scipy.optimize import linprog
import pandas as pd

# -----------------------------
# Configuración
# -----------------------------
url = "https://github.com/santiagonajera/OptimizacionRutas/raw/refs/heads/main/OptimoTransportes2.xlsx"

print("📥 Cargando datos desde el archivo Excel...")

# -----------------------------
# Función para limpiar nombres (quitar espacios, minúsculas)
# -----------------------------
def limpiar_nombre(nombre):
    if isinstance(nombre, str):
        return nombre.strip().replace(" ", "").lower()
    return str(nombre).strip().replace(" ", "").lower()

# -----------------------------
# Cargar datos
# -----------------------------
try:
    # Leer COSTOS
    df_costos = pd.read_excel(url, sheet_name='COSTOS', index_col=0)
    # Limpiar índices y columnas
    centros_raw = df_costos.index.tolist()
    ciudades_raw = df_costos.columns.tolist()

    centros = [limpiar_nombre(c) for c in centros_raw]
    ciudades = [limpiar_nombre(c) for c in ciudades_raw]

    costos = df_costos.values

    # Leer DEMANDA
    df_demanda = pd.read_excel(url, sheet_name='DEMANDA')
    if 'CIUDAD' not in df_demanda.columns or 'DEMANDA' not in df_demanda.columns:
        raise ValueError("Faltan columnas 'CIUDAD' o 'DEMANDA' en hoja DEMANDA")

    demanda_ciudad = df_demanda['CIUDAD'].apply(limpiar_nombre)
    demanda_valores = df_demanda['DEMANDA'].values
    demanda_dict = dict(zip(demanda_ciudad, demanda_valores))

    # Alinear con ciudades en COSTOS
    demanda = np.array([demanda_dict.get(c, 0) for c in ciudades])

    # Leer OFERTA
    df_oferta = pd.read_excel(url, sheet_name='OFERTA')
    if 'CENTRO_DIST' not in df_oferta.columns or 'OFERTA' not in df_oferta.columns:
        raise ValueError("Faltan columnas 'CENTRO_DIST' u 'OFERTA' en hoja OFERTA")

    centro_dist = df_oferta['CENTRO_DIST'].apply(limpiar_nombre)
    oferta_valores = df_oferta['OFERTA'].values
    oferta_dict = dict(zip(centro_dist, oferta_valores))

    # Alinear con centros en COSTOS
    oferta = np.array([oferta_dict.get(c, 0) for c in centros])

    print(f"✅ Datos cargados correctamente.")
    print(f"   → Centros: {centros_raw}")
    print(f"   → Ciudades: {ciudades_raw}")

except Exception as e:
    print(f"❌ Error al cargar los datos: {e}")
    exit()

# -----------------------------
# Validación de tamaño
# -----------------------------
if costos.shape != (len(centros), len(ciudades)):
    print(f"❌ Error: matriz COSTOS es {costos.shape}, esperado ({len(centros)}, {len(ciudades)})")
    exit()

oferta_total = np.sum(oferta)
demanda_total = np.sum(demanda)

print(f"📊 Oferta total: {oferta_total}")
print(f"📊 Demanda total: {demanda_total}")

if not np.isclose(oferta_total, demanda_total):
    print(f"❌ Oferta ≠ Demanda: el problema puede ser infactible.")
    print("💡 Solución: balancear con centro o ciudad ficticia.")
    exit()
else:
    print("✅ Problema balanceado → se puede resolver.")

# -----------------------------
# Resolver problema
# -----------------------------
num_centros = len(centros)
num_ciudades = len(ciudades)
c = costos.flatten()

A_eq = []
b_eq = []

# Oferta
for i in range(num_centros):
    row = np.zeros(num_centros * num_ciudades)
    for j in range(num_ciudades):
        row[i * num_ciudades + j] = 1
    A_eq.append(row)
    b_eq.append(oferta[i])

# Demanda
for j in range(num_ciudades):
    row = np.zeros(num_centros * num_ciudades)
    for i in range(num_centros):
        row[i * num_ciudades + j] = 1
    A_eq.append(row)
    b_eq.append(demanda[j])

bounds = [(0, None)] * (num_centros * num_ciudades)

print("\n🔍 Resolviendo...")
result = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

# -----------------------------
# Mostrar resultados
# -----------------------------
if result.success:
    print("\n✅ Solución encontrada.")
    print(f"💰 Costo mínimo: ${result.fun:,.2f}")

    x = result.x.reshape(num_centros, num_ciudades)

    print(f"\n📦 Envíos (unidades):")
    print(f"{'':<8}" + "".join([f"{ciudad:>10}" for ciudad in ciudades_raw]))
    print("-" * 50)
    for i, centro in enumerate(centros_raw):
        print(f"{centro:<8}" + "".join([f"{x[i,j]:>10.0f}" for j in range(num_ciudades)]))
else:
    print("\n❌ No se encontró solución factible.")
    print(f"🔧 Mensaje: {result.message}")

📥 Cargando datos desde el archivo Excel...
✅ Datos cargados correctamente.
   → Centros: ['CD 1', 'CD 2', 'CD 3']
   → Ciudades: ['Ciudad 1', 'Ciudad 2', 'Ciudad 3', 'Ciudad 4', 'Ciudad 5', 'Ciudad 6', 'Ciudad 7', 'Ciudad 8', 'Ciudad 9', 'Ciudad 10', 'Ciudad 11', 'Ciudad 12', 'Ciudad 13', 'Ciudad 14', 'Ciudad 15', 'Ciudad 16', 'Ciudad 17', 'Ciudad 18', 'Ciudad 19', 'Ciudad 20']
📊 Oferta total: 6100
📊 Demanda total: 6100
✅ Problema balanceado → se puede resolver.

🔍 Resolviendo...

✅ Solución encontrada.
💰 Costo mínimo: $27,100.00

📦 Envíos (unidades):
          Ciudad 1  Ciudad 2  Ciudad 3  Ciudad 4  Ciudad 5  Ciudad 6  Ciudad 7  Ciudad 8  Ciudad 9 Ciudad 10 Ciudad 11 Ciudad 12 Ciudad 13 Ciudad 14 Ciudad 15 Ciudad 16 Ciudad 17 Ciudad 18 Ciudad 19 Ciudad 20
--------------------------------------------------
CD 1           500         0         0         0         0         0         0         0         0         0       200       200       100         0         0         0         0    

In [4]:
import numpy as np
from scipy.optimize import linprog
import pandas as pd

# -----------------------------
# Configuración
# -----------------------------
url = "https://github.com/santiagonajera/OptimizacionRutas/raw/refs/heads/main/OptimoTransportes3.xlsx"

print("📥 Cargando datos desde el archivo Excel...")

# -----------------------------
# Función para limpiar nombres
# -----------------------------
def limpiar_nombre(nombre):
    if isinstance(nombre, str):
        return nombre.strip().replace(" ", "").lower()
    return str(nombre).strip().replace(" ", "").lower()

# -----------------------------
# Cargar datos
# -----------------------------
try:
    # Leer COSTOS
    df_costos = pd.read_excel(url, sheet_name='COSTOS', index_col=0)
    centros_raw = df_costos.index.tolist()
    ciudades_raw = df_costos.columns.tolist()
    centros = [limpiar_nombre(c) for c in centros_raw]
    ciudades = [limpiar_nombre(c) for c in ciudades_raw]
    costos = df_costos.values

    # Leer DEMANDA
    df_demanda = pd.read_excel(url, sheet_name='DEMANDA')
    if 'CIUDAD' not in df_demanda.columns or 'DEMANDA' not in df_demanda.columns:
        raise ValueError("Faltan columnas 'CIUDAD' o 'DEMANDA' en hoja DEMANDA")
    demanda_ciudad = df_demanda['CIUDAD'].apply(limpiar_nombre)
    demanda_valores = df_demanda['DEMANDA'].values
    demanda_dict = dict(zip(demanda_ciudad, demanda_valores))
    demanda = np.array([demanda_dict.get(c, 0) for c in ciudades])

    # Leer OFERTA
    df_oferta = pd.read_excel(url, sheet_name='OFERTA')
    if 'CENTRO_DIST' not in df_oferta.columns or 'OFERTA' not in df_oferta.columns:
        raise ValueError("Faltan columnas 'CENTRO_DIST' u 'OFERTA' en hoja OFERTA")
    centro_dist = df_oferta['CENTRO_DIST'].apply(limpiar_nombre)
    oferta_valores = df_oferta['OFERTA'].values
    oferta_dict = dict(zip(centro_dist, oferta_valores))
    oferta = np.array([oferta_dict.get(c, 0) for c in centros])

    print(f"✅ Datos cargados correctamente.")
    print(f"   → Centros: {centros_raw}")
    print(f"   → Ciudades: {ciudades_raw}")

except Exception as e:
    print(f"❌ Error al cargar los datos: {e}")
    exit()

# -----------------------------
# Validar tamaños
# -----------------------------
if costos.shape != (len(centros), len(ciudades)):
    print(f"❌ Error: matriz COSTOS es {costos.shape}, esperado ({len(centros)}, {len(ciudades)})")
    exit()

oferta_total = np.sum(oferta)
demanda_total = np.sum(demanda)

print(f"📊 Oferta total: {oferta_total}")
print(f"📊 Demanda total: {demanda_total}")

# -----------------------------
# Definir tipo de problema
# -----------------------------
if oferta_total >= demanda_total:
    print("📦 Oferta ≥ Demanda → Enviaremos hasta cubrir demanda. Exceso se almacena.")
    # Restricciones:
    # - Oferta: suma por centro ≤ oferta[i]  → podemos no usar todo
    # - Demanda: suma por ciudad = demanda[j] → cubrimos toda la demanda
    A_ub_oferta = []
    b_ub_oferta = []
    for i in range(len(centros)):
        row = np.zeros(len(centros) * len(ciudades))
        for j in range(len(ciudades)):
            row[i * len(ciudades) + j] = 1
        A_ub_oferta.append(row)
        b_ub_oferta.append(oferta[i])

    A_eq_demanda = []
    b_eq_demanda = []
    for j in range(len(ciudades)):
        row = np.zeros(len(centros) * len(ciudades))
        for i in range(len(centros)):
            row[i * len(ciudades) + j] = 1
        A_eq_demanda.append(row)
        b_eq_demanda.append(demanda[j])

    A_ub = A_ub_oferta
    b_ub = b_ub_oferta
    A_eq = A_eq_demanda
    b_eq = b_eq_demanda

else:  # demanda_total > oferta_total
    print("⚠️ Demanda > Oferta → Cubriremos lo más económico. Habrá faltante.")
    # Restricciones:
    # - Oferta: suma por centro = oferta[i] → usamos toda la oferta
    # - Demanda: suma por ciudad ≤ demanda[j] → no forzamos cubrir todo
    A_eq_oferta = []
    b_eq_oferta = []
    for i in range(len(centros)):
        row = np.zeros(len(centros) * len(ciudades))
        for j in range(len(ciudades)):
            row[i * len(ciudades) + j] = 1
        A_eq_oferta.append(row)
        b_eq_oferta.append(oferta[i])

    A_ub_demanda = []
    b_ub_demanda = []
    for j in range(len(ciudades)):
        row = np.zeros(len(centros) * len(ciudades))
        for i in range(len(centros)):
            row[i * len(ciudades) + j] = 1
        A_ub_demanda.append(row)
        b_ub_demanda.append(demanda[j])

    A_eq = A_eq_oferta
    b_eq = b_eq_oferta
    A_ub = A_ub_demanda
    b_ub = b_ub_demanda

# -----------------------------
# Resolver problema
# -----------------------------
c = costos.flatten()
bounds = [(0, None)] * (len(centros) * len(ciudades))

print("\n🔍 Resolviendo...")
result = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

# -----------------------------
# Mostrar resultados
# -----------------------------
if result.success:
    print("\n✅ Solución encontrada.")
    print(f"💰 Costo total: ${result.fun:,.2f}")

    x = result.x.reshape(len(centros), len(ciudades))

    print(f"\n📦 Envíos realizados (unidades):")
    print(f"{'':<8}" + "".join([f"{ciudad:>10}" for ciudad in ciudades_raw]))
    print("-" * 50)
    for i, centro in enumerate(centros_raw):
        print(f"{centro:<8}" + "".join([f"{x[i,j]:>10.0f}" for j in range(len(ciudades))]))

    # Calcular cuánto se envió por ciudad
    enviado = x.sum(axis=0)
    faltante = np.maximum(0, demanda - enviado)

    # Calcular cuánto se almacenó por centro
    usado = x.sum(axis=1)
    almacenado = np.maximum(0, oferta - usado)

    # Reportar
    if oferta_total >= demanda_total:
        print("\n✅ Toda la demanda fue cubierta.")
        print("📦 Exceso de oferta almacenado por centro:")
        for i, centro in enumerate(centros_raw):
            if almacenado[i] > 0:
                print(f"   → {centro}: {almacenado[i]:.0f} unidades almacenadas")
    else:
        print("\n⚠️ No se pudo cubrir toda la demanda. Faltante por ciudad:")
        for j, ciudad in enumerate(ciudades_raw):
            if faltante[j] > 0:
                print(f"   → {ciudad}: {faltante[j]:.0f} unidades faltantes")

else:
    print("\n❌ No se encontró solución factible.")
    print(f"🔧 Mensaje: {result.message}")

📥 Cargando datos desde el archivo Excel...
✅ Datos cargados correctamente.
   → Centros: ['CD 1', 'CD 2', 'CD 3']
   → Ciudades: ['Ciudad 1', 'Ciudad 2', 'Ciudad 3', 'Ciudad 4', 'Ciudad 5', 'Ciudad 6', 'Ciudad 7', 'Ciudad 8', 'Ciudad 9', 'Ciudad 10', 'Ciudad 11', 'Ciudad 12', 'Ciudad 13', 'Ciudad 14', 'Ciudad 15', 'Ciudad 16', 'Ciudad 17', 'Ciudad 18', 'Ciudad 19', 'Ciudad 20']
📊 Oferta total: 5600
📊 Demanda total: 6100
⚠️ Demanda > Oferta → Cubriremos lo más económico. Habrá faltante.

🔍 Resolviendo...

✅ Solución encontrada.
💰 Costo total: $22,800.00

📦 Envíos realizados (unidades):
          Ciudad 1  Ciudad 2  Ciudad 3  Ciudad 4  Ciudad 5  Ciudad 6  Ciudad 7  Ciudad 8  Ciudad 9 Ciudad 10 Ciudad 11 Ciudad 12 Ciudad 13 Ciudad 14 Ciudad 15 Ciudad 16 Ciudad 17 Ciudad 18 Ciudad 19 Ciudad 20
--------------------------------------------------
CD 1           500         0         0         0         0         0         0         0       100         0       200       200         0         0